In [ ]:
# Install dependencis in your terminal
# pip install mediapipe opencv-python


In [2]:
# Import all needed packages
import mediapipe as mp
import cv2
import numpy as np
import os
import uuid
import math
from interface import *

In [3]:
# Mediapipe confg variables
mp_drawing = mp.solutions.drawing_utils # Module for rendering the landmarks on video/image
mp_hands = mp.solutions.hands # Module for work with hands - detection and tracking 

In [4]:
# Active WebCam
cap = cv2.VideoCapture(0)
img_height = cap.get(4)
img_width = cap.get(3)

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence= 0.5) as hands:
    while cap.isOpened():
        _, frame = cap.read() # Catches the frame from video

        # Hands detections
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # Change frame's colors into RGB scheme
        image.flags.writeable = False # Set flag to false
        results = hands.process(image)  # Detections
        image.flags.writeable = True # Set flagto true
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # Rechange colors
        
        # Draw detections on frame
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS)
            try:
                coord_list = add_to_list(coord_list, hand)
            except NameError:
                coord_list = [[hand.landmark[8].x, hand.landmark[8].y]]
                first_item = coord_list[0] 
                 
            print(coord_list)

            cv2.line(image, denormalization(hand, 8, img_height, img_width), denormalization(hand, 4, img_height, img_width), (255, 0, 0), 3)

        cv2.imshow('Hand Tracking', cv2.flip(image,1))  # Display frame; Flip display image as mirrored

        # Get last 8 bits from pressed key.
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release() # Release WebCam
    cv2.destroyAllWindows() 



[[0.5957105755805969, 0.6803699135780334]]
[[0.5957105755805969, 0.6803699135780334], [0.5916463732719421, 0.678231954574585]]
[[0.5957105755805969, 0.6803699135780334], [0.5916463732719421, 0.678231954574585], [0.5912320613861084, 0.6807281970977783]]
[[0.5957105755805969, 0.6803699135780334], [0.5916463732719421, 0.678231954574585], [0.5912320613861084, 0.6807281970977783], [0.5904721617698669, 0.683022677898407]]
[[0.5957105755805969, 0.6803699135780334], [0.5916463732719421, 0.678231954574585], [0.5912320613861084, 0.6807281970977783], [0.5904721617698669, 0.683022677898407], [0.5913506150245667, 0.6835642457008362]]
[[0.5957105755805969, 0.6803699135780334], [0.5916463732719421, 0.678231954574585], [0.5912320613861084, 0.6807281970977783], [0.5904721617698669, 0.683022677898407], [0.5913506150245667, 0.6835642457008362], [0.5890060067176819, 0.6819030046463013]]
[[0.5957105755805969, 0.6803699135780334], [0.5916463732719421, 0.678231954574585], [0.5912320613861084, 0.6807281970977

In [5]:
print(first_item)

[0.5957105755805969, 0.6803699135780334]
